<a href="https://colab.research.google.com/github/LadaOndris/IBT/blob/master/datasets/handseg150k/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
def load_images(start_index = 0, end_index = 99):
  all_images_paths = glob.glob("/content/drive/My Drive/handseg150k/images/*")
  
  if (start_index != None and end_index != None):
    images_paths = all_images_paths[start_index:end_index]
  elif (start_index == None and end_index != None):
    images_paths = all_images_paths[:end_index]
  elif (start_index != None and end_index == None):
    images_paths = all_images_paths[start_index:]
  else:
    images_paths = all_images_paths

  images = [np.array(Image.open(filename)) for filename in images_paths]
  for image in images:
    image[image == 0] = HUGE_INT
          
  print("Loaded images:", np.array(images).shape)


100
(100, 480, 640)


In [13]:
print(len(images))

100


Compute features for given images:


In [20]:
HUGE_INT = 2147483647

class RdfFeatures:

    def __init__(self):
        self.image_shape = (480, 640)
        self.offsets = None
        self.pixels = None
        return

    def get_random_pixels(self, count, image_shape):
        return [[i, j] for i in range(0, 480, 12) for j in range(0, 640, 12)]

    def get_offset(self, count, image_shape):
        half_width = image_shape[0] / 2.0
        half_height = image_shape[1] / 2.0
        x = np.random.randint(-half_width, half_width, count, dtype=np.int32) 
        y = np.random.randint(-half_height, half_height, count, dtype=np.int32) 
        return np.column_stack((x, y))
    
    def get_offsets(self, count, image_shape):
        u = self.get_offset(count, image_shape)
        v = self.get_offset(count, image_shape)
        return np.dstack((u, v))
    
    def get_depth(self, image, coord):
        if (coord[0] >= 0 and coord[1] >= 0 and
            coord[0] < image.shape[0] and
            coord[1] < image.shape[1]):
            return image[coord[0], coord[1]]
        return HUGE_INT
    
    def get_depth_m(self, image, coords):
        depths = np.full(shape=(len(coords)), fill_value=HUGE_INT, dtype=int)
        mask = (coords[:,0] < image.shape[0]) & (coords[:,1] < image.shape[1]) & \
            (coords[:,0] >= 0) & (coords[:,1] >= 0)
        valid = coords[mask]
        depths[mask] = image[valid[:, 0], valid[:, 1]]
        return depths
    
    def calculate_feature(self, image, pixel, u, v):
        pixelDepth = image[pixel[0], pixel[1]]#get_depth(image, pixel)
        u = np.divide(u * 10000, pixelDepth).astype(int)
        v = np.divide(v * 10000, pixelDepth).astype(int)
        p1 = np.add(pixel, u)
        p2 = np.add(pixel, v)
        return self.get_depth(image, p1) - self.get_depth(image, p2)
    
    def get_features_for_pixel(self, image, pixel, offsets):
        features = np.array([])
        for u, v in offsets:
            feature = self.calculate_feature(image, pixel, u, v)
            features = np.insert(features, len(features), feature)
            #if feature != 0 and feature < 2147000000 and feature > -2147000000:
            #    print(feature)
            
        return features
    
    def get_features_for_pixel_m(self, image, pixel):
        u = self.offsets[:,0]
        v = self.offsets[:,1]
        pixelDepth = image[pixel[0], pixel[1]]
        u = np.divide(u * 10000, pixelDepth).astype(int)
        v = np.divide(v * 10000, pixelDepth).astype(int)
        p1 = np.add(pixel, u)
        p2 = np.add(pixel, v)
        return np.subtract(self.get_depth_m(image, p1), self.get_depth_m(image, p2))
        
    
    def get_label(self, mask, pixel):
        value = mask[pixel[0], pixel[1]]
        if value == 0:
            return 0
        return 1
        
    """
    Computes features and labels for the given image.
    """
    def get_samples_for_image(self, image, mask):
        features = np.empty(shape=(len(self.pixels), len(self.offsets)), dtype=int)
        labels = np.empty(shape=(len(self.pixels)), dtype=int)
        for i, pixel in enumerate(self.pixels):
            features[i] = self.get_features_for_pixel_m(image, pixel)
            labels[i] = self.get_label(mask, pixel)
        return features, labels
            
    """
    Returns a tuple consisting of features and corresponding labels.
    
    Attributes:
        num_images: number of images to extract the features from
        sampled_pixels_count: Number of pixels to be randomly selected and 
            used to extract features from.
            It is not used if pixels parameter is given.
        total_features: Number of feature to be extracted for each pixel.
            It is not used if offsets parameter is given.
        #pixels: List of coordinates, target pixels, to extract the features 
        #    from.
        #offsets: Offsets which are used to calculate features from a pixel. 
        #    For each feature, there is a tuple of two offsets.
    """
    def get_samples(self,
                    image_start_index = 0,
                    image_end_index = 99,
                    sampled_pixels_count = 2000, 
                    total_features = 2000):
        #images, masks = self.load_images(image_start_index, image_end_index)
        images_subset = images[image_start_index : image_end_index]

        if self.offsets is None:
            self.offsets = self.get_offsets(total_features, self.image_shape)
        if self.pixels is None:
            self.pixels = self.get_random_pixels(sampled_pixels_count, self.image_shape)
        
        num_images = image_end_index - image_start_index + 1
        features = np.ndarray(shape=(num_images * len(self.pixels), len(self.offsets)))
        #labels = np.ndarray(shape=(num_images * len(self.pixels)))
        
        for i, image in enumerate(images_subset):
            for p, pixel in enumerate(self.pixels):
                #if image[pixel[0], pixel[1]] == HUGE_INT:
                #    continue
                features[i * len(self.pixels) + p] = self.get_features_for_pixel_m(image, pixel)
                #labels[i * len(self.pixels) + p] = self.get_label(mask, pixel)
            
        #print(sum([l for l in labels if l == 1]))
        return features#, labels


In [22]:
f = RdfFeatures()
features = f.get_samples(0, 99)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide
